<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Monte Carlo simulations
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook shows the results from the simulation study
</p>

</div>


## 1. Loading Packages and data


In [3]:
from simulations.simulation_functions import simulate, Surface
import numpy as np
import plotly.graph_objects as go
from utils import create_pred_input

2025-08-13 13:22:40.032257: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-13 13:22:40.514248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 13:22:45.416955: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-13 13:22:50.741684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755091375.407344   18750 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755091376.27

In [4]:

def calculate_surface(specification, element, date):
    data= simulate(seed=0, n_countries=196, n_years=63, specification=specification, add_noise=True)
    
    
    _, T, P = create_pred_input(
        True,
        mean_T=np.mean(data["temperature"]),
        std_T=np.std(data["temperature"]),
        mean_P=np.mean(data["precipitation"]),
        std_P=np.std(data["precipitation"])
    )
    
    surface=np.load(f"../results/MonteCarlo/{specification}/{date}/_avg_surface.np.npy")
    model_values = np.mean(surface, axis=0).reshape(T.shape)
    benchmark = Surface(T.flatten(), P.flatten(), specification=specification).reshape(T.shape)

    
    if element == 'model':
    
        surf = go.Surface(
            x=T, y=P, z=model_values,
            colorscale='Plasma',
            opacity=0.85,
            showscale=False,
            name=f'Model'
        )
    
    elif element == 'benchmark':
        surf= go.Surface(
            x=T, y=P, z=benchmark,
            colorscale='Plasma',
            opacity=0.85,
            showscale=False,
            name=f'Benchmark'
        )
    
    else: 
        bias=model_values-benchmark
        surf=go.Heatmap(
        x=P[:, 0], y=T[0, :], z=bias,
        colorscale='RdYlBu', zmin=-0.2, zmax=0.2,
        showscale=True,
        colorbar=dict(
            title='Bias', thickness=15, len=1
        )
        )
    fig=create_figure(element, surf)
    return fig


def create_figure(element, plot_data):
    
    if element=="model" or element=="benchmark":
        fig = go.Figure(data=plot_data)
        fig.update_layout(
            scene=dict(
                xaxis_title='Temperature (°C)',
                yaxis_title='Precipitation (m)',
                zaxis_title='Δ ln(Growth)',
                
                camera=dict(eye=dict(x=2.11, y=0.12, z=0.38))
            ),
            legend=dict(
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='black',
                borderwidth=1
            )
        )
    else: 
        fig = go.Figure(data=plot_data)
        fig.update_layout(
        xaxis=dict(title='Precipitation (m)'),
        yaxis=dict(title='Temperature (°C)'),
        margin=dict(l=60, r=20, t=40, b=60)
    )

    return fig


In [5]:
fig=calculate_surface("q_Leirvik", "model", "2025-07-11")
fig.show()


In [ ]:
data= simulate(seed=0, n_countries=196, n_years=63, specification='q_Leirvik', add_noise=True)


_, T, P = create_pred_input(
    True,
    mean_T=np.mean(data["temperature"]),
    std_T=np.std(data["temperature"]),
    mean_P=np.mean(data["precipitation"]),
    std_P=np.std(data["precipitation"])
)

surface=np.load(f"../results/MonteCarlo/q_Leirvik/2025-07-11/_avg_surface.np.npy")

model_values = np.mean(surface, axis=0).reshape(T.shape)


print(T.shape)

In [12]:
print(model_values)




[[ 0.03002555  0.03209399  0.03406715 ... -0.13265428 -0.13987911
  -0.14725275]
 [ 0.03072958  0.03279156  0.03475266 ... -0.13318938 -0.14035043
  -0.14765874]
 [ 0.03135031  0.03341142  0.03536596 ... -0.1336544  -0.14075056
  -0.14799225]
 ...
 [-1.2265807  -1.1945752  -1.1627647  ... -0.19004638 -0.1992814
  -0.20873217]
 [-1.2557058  -1.2231244  -1.1907338  ... -0.19110951 -0.20036113
  -0.20983227]
 [-1.2848378  -1.2516897  -1.2187284  ... -0.19226627 -0.20152873
  -0.21101454]]


In [14]:
#optionally save as pdf 
fig.write_image("../results/images/Monte Carlo/q_Leirvik_bias.pdf")